<a href="https://colab.research.google.com/github/gauravvxv/QuickBite-Express-Crisis-Recovery/blob/main/Notebook/QuickBite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **QuickBite Express Crisis Recovery**


## **Business Problem**

QuickBite Express is a Bengaluru-based food-tech startup (founded in 2020) that connects customers with nearby restaurants and cloud kitchens. In June 2025, QuickBite faced a major crisis. A viral social media incident involving food safety violations at partner restaurants, combined with a week-long delivery outage during the monsoon season, triggered massive customer backlash. Competitors capitalized with aggressive campaigns, worsening the situation.

The challenges were severe:

* A large portion of active users disengaged within a short period.
* Daily orders saw a sharp decline compared to earlier months.
* Customer satisfaction scores fell sharply, signaling trust issues.
* Many partner restaurants shifted to competing platforms.
* Customer acquisition costs rose significantly.

## **Import Libraries**

In [47]:
import pandas as pd

## **Load CSV file**

In [42]:
customers = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/dim_customer.csv')
d_partner = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/dim_delivery_partner_.csv')
menu = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/dim_menu_item.csv')
restaurants = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/dim_restaurant.csv')
d_performance = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/fact_delivery_performance.csv')
order_items = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/fact_order_items.csv')
orders = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/fact_orders.csv')
rating = pd.read_csv('/content/drive/MyDrive/RPC_18_Datasets/fact_ratings.csv')

## **All DataFrame into one Dictionary**

In [43]:
datasets = {
    'customers': customers,
    'delivery_partner': d_partner,
    'menu': menu,
    'restaurants': restaurants,
    'delivery_performance': d_performance,
    'order_items': order_items,
    'orders': orders,
    'rating': rating
}

In [44]:
summary = []

for name, df in datasets.items():
    row = {
        'Dataset': name,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Total missing values': df.isnull().sum().sum(),
        'Total duplicate rows': df.duplicated().sum()
    }
    summary.append(row)

summary_df = pd.DataFrame(summary)

summary_df

,Dataset,Rows,Columns,Total missing values,Total duplicate rows
0,customers,107776,4,0,0
1,delivery_partner,15000,7,0,0
2,menu,342671,6,0,0
3,restaurants,19995,7,0,0
4,delivery_performance,149166,4,0,0
5,order_items,342994,8,0,0
6,orders,149166,11,5635,0
7,rating,68842,7,119,16


In [45]:
for name, df in datasets.items():
    print(f"Null values in dataset: {name}")
    null_counts = df.isnull().sum()
    null_cols = null_counts[null_counts > 0]

    if null_cols.empty:
        print("No null values found.")
    else:
        print(null_cols)

    print("\n")

Null values in dataset: customers
No null values found.


Null values in dataset: delivery_partner
No null values found.


Null values in dataset: menu
No null values found.


Null values in dataset: restaurants
No null values found.


Null values in dataset: delivery_performance
No null values found.


Null values in dataset: order_items
No null values found.


Null values in dataset: orders
delivery_partner_id    5635
dtype: int64


Null values in dataset: rating
order_id            17
customer_id         17
restaurant_id       17
rating              17
review_text         17
review_timestamp    17
sentiment_score     17
dtype: int64




In [46]:
rating.dropna(inplace=True)

In [52]:
customers.head()

,customer_id,signup_date,city,acquisition_channel
0,CUST000007,21-03-2025,Pune,Organic
1,CUST000008,07-02-2025,Kolkata,Referral
2,CUST000009,25-02-2025,Delhi,Paid
3,CUST000010,28-01-2025,Hyderabad,Paid
4,CUST000011,18-01-2025,Kolkata,Organic


In [53]:
orders.head()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N
3,ORD202501000124,CUST191820,REST19745,DP13859,2025-01-01 12:00:00,260.81,0.00,32.75,293.56,N,N
4,ORD202501006518,CUST033760,REST12962,DP09615,2025-01-01 12:00:00,280.33,0.00,25.57,305.90,N,N


In [54]:
order_items.head()

,order_id,item_id,menu_item_id,restaurant_id,quantity,unit_price,item_discount,line_total
0,ORD202501006518,ITEM001,MENU12962_3216,REST12962,2,48.31,0.00,96.62
1,ORD202501006518,ITEM002,MENU12962_1962,REST12962,3,61.24,0.00,183.71
2,ORD202501019281,ITEM001,MENU14069_0510,REST14069,2,87.19,0.00,174.38
3,ORD202501019281,ITEM002,MENU14069_4308,REST14069,3,84.67,0.00,254.00
4,ORD202501023439,ITEM001,MENU08622_7574,REST08622,2,235.81,35.44,436.18


In [55]:
d_partner.head()

,delivery_partner_id,partner_name,city,vehicle_type,employment_type,avg_rating,is_active
0,DP09615,Neha E,Bengaluru,Scooter,Full-time,3.77,Y
1,DP02021,Neha V,Bengaluru,Bike,Full-time,3.90,Y
2,DP05541,Vikas S,Delhi,Cycle,Part-time,3.33,Y
3,DP13859,Amit B,Delhi,Cycle,Part-time,4.07,N
4,DP08091,Arjun R,Mumbai,Car,Full-time,4.63,Y


In [56]:
menu.head()

,menu_item_id,restaurant_id,item_name,category,is_veg,price
0,MENU12962_3216,REST12962,Paneer Tikka Pizza,Pizza,Y,271.05
1,MENU12962_1962,REST12962,Pepperoni Pizza,Pizza,N,327.26
2,MENU14069_0510,REST14069,Egg Fried Rice,Fried Rice,N,200.01
3,MENU14069_4308,REST14069,Paneer Tikka,Starters,Y,198.83
4,MENU08622_7574,REST08622,Kadai Paneer,Curries,Y,271.60


In [57]:
restaurants.head()

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
0,REST12962,Flavours of Sweets Palace,Bengaluru,Chinese,Restaurant,26-40,N
1,REST14069,Royal Biryani Darbar,Ahmedabad,Fast Food,Restaurant,16-25,Y
2,REST08622,Spicy Wraps Point,Mumbai,North Indian,Restaurant,16-25,Y
3,REST19745,Delhi Express Palace,Ahmedabad,Pizza,Cloud Kitchen,16-25,Y
4,REST02383,Namma Mess Delight,Mumbai,Healthy,Restaurant,26-40,Y


In [58]:
d_performance.head()

,order_id,actual_delivery_time_mins,expected_delivery_time_mins,distance_km
0,ORD202501023439,31,31,6.4
1,ORD202501012051,46,42,1.9
2,ORD202501019281,25,31,6.2
3,ORD202501000124,29,41,6.2
4,ORD202501006518,37,41,2.4


In [59]:
rating.head()

,order_id,customer_id,restaurant_id,rating,review_text,review_timestamp,sentiment_score
0,ORD202501023439,CUST181110,REST08622,4.5,Super fast delivery,01-01-2025 15:00,0.75
1,ORD202501019281,CUST179306,REST14069,4.5,Great taste!,01-01-2025 15:00,0.75
2,ORD202501018036,CUST093042,REST13907,5.0,Super fast delivery,01-01-2025 14:03,1.00
3,ORD202501007724,CUST110825,REST08451,4.0,Tasty but a bit late,01-01-2025 14:06,0.50
4,ORD202501002349,CUST181669,REST00225,4.4,Satisfied overall,01-01-2025 14:06,0.70
